In [1]:
import pandas as pd

In [2]:
df_eleitorados = pd.read_csv("eleitorado/perfil_eleitorado_2020/perfil_eleitorado_2020.csv", encoding="Latin 1", quotechar='"', sep=";"  )

In [3]:
df_resultados = pd.read_csv("resultados/SP_turno_1.csv", encoding="Latin 1", on_bad_lines='skip', quotechar='"', sep=";")

### Tratar Dados

In [4]:
col_nao_necessarias_eleitorados = ["DT_GERACAO", "HH_GERACAO", "ANO_ELEICAO", "CD_MUN_SIT_BIOMETRIA", "DS_MUN_SIT_BIOMETRIA", "QT_ELEITORES_BIOMETRIA", "QT_ELEITORES_DEFICIENCIA", "QT_ELEITORES_INC_NM_SOCIAL", "SG_UF"]
col_nao_necessarias_resultados = ["DT_GERACAO", "HH_GERACAO", "ANO_ELEICAO", "CD_TIPO_ELEICAO", "NM_TIPO_ELEICAO", "DT_PLEITO", "NR_TURNO", "DS_ELEICAO", "SG_UF", "DS_CARGO_PERGUNTA", "DT_BU_RECEBIDO", "QT_ABSTENCOES", "CD_TIPO_URNA", "DS_TIPO_URNA", "NR_URNA_EFETIVADA", "CD_CARGA_1_URNA_EFETIVADA", "CD_CARGA_2_URNA_EFETIVADA", "CD_FLASHCARD_URNA_EFETIVADA", "DT_CARGA_URNA_EFETIVADA", "DS_AGREGADAS", "DT_ABERTURA", "DT_ENCERRAMENTO", "QT_ELEITORES_BIOMETRIA_NH", "NR_JUNTA_APURADORA", "NR_TURMA_APURADORA", "DT_EMISSAO_BU"]


In [5]:
for col in col_nao_necessarias_eleitorados:
    df_eleitorados = df_eleitorados.drop(col, axis=1)

In [6]:
for col_ in col_nao_necessarias_resultados:
    df_resultados = df_resultados.drop(col_, axis=1)

### Query 1 - Em qual município o candidato X foi mais votado;


In [7]:
# Selecionar o candidato Bruno Covas como exemplo
candidato_x = "BRUNO COVAS"
candidato_df = df_resultados[df_resultados['NM_VOTAVEL'] == candidato_x]

#Encontrar o município que mais votou no candidato
municipio_mais_votado = candidato_df.groupby('NM_MUNICIPIO')['QT_VOTOS'].max().reset_index()
print(municipio_mais_votado)

#Comentário: acredito que o número de votos está errado, pois 171 é um valor muito baixo. 
#O valor em si é de apenas uma row, não consegui achar outra maneira a tempo de somar quantidade de votos. 

  NM_MUNICIPIO  QT_VOTOS
0    SÃO PAULO       171


### Query 2 - Qual candidato foi mais votado em cada município;

In [8]:
# Filtrar apenas os registros relevantes da base de dados de resultados
resultados_relevantes = df_resultados[['CD_MUNICIPIO', 'NM_MUNICIPIO', 'NM_VOTAVEL', 'QT_VOTOS']]

# Encontrar o candidato mais votado em cada município
candidato_mais_votado_por_municipio = resultados_relevantes.groupby(['CD_MUNICIPIO', 'NM_MUNICIPIO'])['QT_VOTOS'].idxmax()
candidatos_mais_votados = resultados_relevantes.loc[candidato_mais_votado_por_municipio][['CD_MUNICIPIO', 'NM_MUNICIPIO', 'NM_VOTAVEL', 'QT_VOTOS']]

# Exibir o resultado
print("Candidato mais votado em cada município:")
print(candidatos_mais_votados)

#Comentário: mesma situação da query 1, os números de votos estão baixos

Candidato mais votado em cada município:
         CD_MUNICIPIO                NM_MUNICIPIO     NM_VOTAVEL  QT_VOTOS
6173488         61000  EUCLIDES DA CUNHA PAULISTA     NENE LOPES       237
2580623         61018                  ADAMANTINA  MARCIO CARDIM       254
2621902         61026                      BOREBI      CHIQUINHO       300
1266714         61034                      ADOLFO           CHEM       228
5366025         61042                      TARUMÃ          OSCAR       183
...               ...                         ...            ...       ...
1760317         72397                   VIRADOURO    CAL RIBEIRO       219
1528679         72419        VISTA ALEGRE DO ALTO   LUIS FIORANI       227
3510441         72435                  VOTORANTIM  FABIOLA ALVES       106
2497643         72451                 VOTUPORANGA     JORGE SEBA       158
5283248         72737      VARGEM GRANDE PAULISTA    JOSUÉ RAMOS       218

[645 rows x 4 columns]


### Query 3 - Qual perfil do eleitorado (faixa etária, gênero, grau de escolaridade, etc.) mais votou em cada candidato;


In [10]:
# Definir amostragem
porcentagem_amostra = 0.003

# Realizar a amostragem aleatória
df_resultados = df_resultados.sample(frac=porcentagem_amostra, random_state=42)

# Filtrar apenas os registros relevantes da base de dados de resultados
resultados_relevantes = df_resultados[['NM_VOTAVEL', 'NM_MUNICIPIO', 'QT_VOTOS']]

# Juntar com as informações da base de dados de eleitorados para obter os detalhes do perfil do eleitorado
resultados_com_perfil = pd.merge(resultados_relevantes, df_eleitorados, left_on='NM_MUNICIPIO', right_on='NM_MUNICIPIO', how='inner')

# Agrupar os resultados por candidato e perfil do eleitorado, calculando a quantidade total de votos por perfil
perfil_eleitorado_mais_votou = resultados_com_perfil.groupby(['NM_VOTAVEL', 'DS_GENERO', 'DS_FAIXA_ETARIA', 'DS_GRAU_ESCOLARIDADE'])['QT_VOTOS'].sum().reset_index()

# Para cada candidato, encontrar o perfil do eleitorado que mais votou
perfil_mais_votou_por_candidato = perfil_eleitorado_mais_votou.groupby('NM_VOTAVEL', as_index=False).apply(lambda group: group.loc[group['QT_VOTOS'].idxmax()])

# Exibir o resultado
print("Perfil do eleitorado que mais votou em cada candidato:")
print(perfil_mais_votou_por_candidato[['NM_VOTAVEL', 'DS_GENERO', 'DS_FAIXA_ETARIA', 'DS_GRAU_ESCOLARIDADE', 'QT_VOTOS']])

#Comentário: na query 3 pode-se observar que o resultado ficou divido em duas tabelas, não consegui dar merge nas duas.

Perfil do eleitorado que mais votou em cada candidato:
                      NM_VOTAVEL  DS_GENERO                 DS_FAIXA_ETARIA  \
0    ADRIANA GRIFONI DA EDUCAÇÃO   FEMININO  70 a 74 anos                     
1                AILTON DO GESSO   FEMININO  70 a 74 anos                     
2            ALEXANDRE RODRIGUES  MASCULINO  50 a 54 anos                     
3                    ALINE AMARO   FEMININO  55 a 59 anos                     
4              ALTAIR DA PADARIA  MASCULINO  75 a 79 anos                     
..                           ...        ...                             ...   
106                   VERA FARIA   FEMININO  70 a 74 anos                     
107                VINÍCIUS LIMA   FEMININO  75 a 79 anos                     
108               VIVIAN PELODAN   FEMININO  80 a 84 anos                     
109                         ZEMA   FEMININO  30 a 34 anos                     
110       ÁDAMO ALVES RADIALISTA   FEMININO  55 a 59 anos                   